In [21]:
def findPath(stream_1,stream_2):
    path =[]
    for SLA in client_SLAs :
        if stream_1.addrDest in SLA.clientNetwok :
            path.append(SLA) 
        if stream_2.addrDest in SLA.clientNetwok :
            path.append(SLA) 
    return path

class Resa :
    id = None
    stream_1 = None
    stream_2 = None
    status = None
    path = None
    capacity = 0

    # paths = None
    # Maybe we need to have multiple path if the shorter one can't meet QoS need

    def __init__(self, id , steam1, stream2):
        self.id = id
        self.stream_1 = steam1
        self.stream_2 = stream2
        self.status = 'Unactive'
        self.path = findPath(self.stream_1,self.stream_2)
        # self.paths = findAllPath(self.stream_1,self.stream_2)

    def askResa(self, tspec):
        resaOK = True

        # Check if all SLA in path have the capacity
        for SLA in self.path :
            if not SLA.checkCapacity(tspec) :
                resaOK = False 
                break
        # Book require capacity on evry SLA in path
        if resaOK :
            for SLA in self.path :
                SLA.book(self.stream_1,self.stream_2, tspec)
        self.status = 'Active'
        self.capacity += tspec
        return resaOK
    def cancelResa(self):
        # Unbook require capacity on evry SLA in path
        for SLA in self.path :
            SLA.unBook(self.stream_1,self.stream_2, self.capacity)
            self.capacity = 0
        self.status = 'Unactive'

SyntaxError: invalid syntax (1352184789.py, line 7)

In [22]:
class Steam :
    portDest = None
    addrDest = None
    portSrc = None
    addrSrc = None
    protocol = 'UDP'
    codec = 'Defaul'
    params = None

    def __init__(self,portDest, addrDest, portSrc, addrSrc,protocol='UDP',codec='Default', params=None):
        self.portDest = portDest
        self.addrDest = addrDest
        self.portSrc = portSrc
        self.addrSrc = addrSrc
        self.protocol = protocol
        self.codec = codec
        self.params = params


In [13]:
class SLA :

    id = None
    CE :CE = None
    clientNetwork = None
    capacity = None
    usage = 0

    def __init__(self,id, CE, clientNetwork, capacity): 
        self.id = id
        self.CE = CE
        self.clientNetwork = clientNetwork
        self.capacity = capacity

    def checkCapacity(self ,tspec) :
        result = False
        if self.capacity > (self.usage + tspec):
            result = True
        return result

    def book(self, stream_1, stream_2, tspec):
        self.usage += tspec
        self.CE.book(stream_1,stream_2, tspec)

    def unBook(self, stream_1, stream_2, tspec):
        self.usage -= tspec
        self.CE.unBook(stream_1,stream_2, tspec)

In [4]:
from netmiko import ConnectHandler

def getBookingConfig(stream_1,stream_2,tspec):
    config_commands =[]
    # TO DO !
    return config_commands

def getUnBookingConfig(stream_1,stream_2,tspec):
    config_commands =[]
    # TO DO !
    return config_commands
    

class CE :
    id = None
    device = {
        'device_type': 'cisco_ios',
        'host':   '10.10.10.10',
        'username': 'test',
        'password': 'password',
        'port' : 8022,          # optional, defaults to 22
        'secret': 'secret',     # optional, defaults to ''
    }
    
    def __init__(self,id, device):
        self.id = id
        self.device = device

    def book(self, stream_1,stream_2,tspec) :
        net_connect = ConnectHandler(**self.device)
        config_commands = getBookingConfig(stream_1,stream_2,tspec)
        net_connect.send_config_set(config_commands)

    def unBook(self, stream_1,stream_2,tspec) :
        net_connect = ConnectHandler(**self.device)
        config_commands = getUnBookingConfig(stream_1,stream_2,tspec)
        net_connect.send_config_set(config_commands)
        

In [20]:

from ipaddress import IPv4Network


# Create 3 CE
CE1= CE (
    id='CE1',
    device ={
        'device_type': 'cisco_ios',
        'host':   '193.168.1.11',
        'username': 'test',
        'password': 'password',
        'port' : 8022,
        'secret': 'secret',
    }
)
CE2= CE (
    id='CE2',
    device ={
        'device_type': 'cisco_ios',
        'host':   '193.168.2.22',
        'username': 'test',
        'password': 'password',
        'port' : 8022,
        'secret': 'secret',
    }
)
CE3= CE (
    id='CE3',
    device ={
        'device_type': 'cisco_ios',
        'host':   '193.168.3.33',
        'username': 'test',
        'password': 'password',
        'port' : 8022,
        'secret': 'secret',
    }
)

# Create 3 SLA 
SLA1 = SLA(id='SLA1', clientNetwork=IPv4Network('192.168.1.0/24'), CE=CE1, capacity=1000)
SLA2 = SLA(id='SLA2', clientNetwork=IPv4Network('192.168.2.0/24'), CE=CE2, capacity=1000)
SLA3 = SLA(id='SLA3', clientNetwork=IPv4Network('192.168.3.0/24'), CE=CE3, capacity=1000)

client_SLAs=[SLA1,SLA2,SLA3]
# while True :
print(SLA1.checkCapacity(1001))

<class 'str'>
False
